# Baseline

### **Загрузка библиотек**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

## **Загрузка данных**

In [ ]:
train_df = pd.read_parquet("/kaggle/input/alphadataset/train_data.pqt")
test_df = pd.read_parquet("/kaggle/input/alphadataset/test_data.pqt")

In [ ]:
train_df.head(3)

In [ ]:
test_df.head(3)

## **Предобработка данных**

Определение списка категориальных признаков:

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

Выделение числовых признаков:

In [ ]:
non_cat_cols = train_df.drop(cat_cols, axis=1)

### **Преобразование категориальных признаков**:
   - Приведение значений в категориальных признаках к типу данных "str" (строка) как в обучающем наборе данных train_df, так и в тестовом наборе данных test_df.
   - Для обучения модели, в дальнейшем категориальные признаки были преобразованы в тип "category" для корректной работы модели CatBoost.



In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("str")
test_df[cat_cols] = test_df[cat_cols].astype("str")

### **Создаем выборки для валидации и обучения**

 **Добавление данных за предыдущие месяцы**:
   - Для учета динамики временных данных, были созданы дополнительные столбцы, содержащие информацию за предыдущие два месяца(1 и 2).
   - Данные за предыдущие месяцы были объединены с данными за последний месяц на основе уникального идентификатора.

Оставляем только 3 месяц, данные за 1 и 2 добаляем как новые столбцы

In [ ]:
import pandas as pd


# Фильтрация данных, оставляем только записи за последний месяц
last_month_df = train_df[train_df['date'] == 'month_3']

# Создание DataFrame для данных за предыдущие два месяца
prev_month_2_df = train_df[train_df['date'] == 'month_2'].set_index('id')
prev_month_1_df = train_df[train_df['date'] == 'month_1'].set_index('id')
prev_month_2_df[cat_cols] = prev_month_2_df[cat_cols].astype("category")
prev_month_1_df[cat_cols] = prev_month_1_df[cat_cols].astype("category")

# Переименование столбцов, чтобы избежать конфликтов имен
prev_month_2_df.columns = [f'{col}_prev_month_2' for col in prev_month_2_df.columns]
prev_month_1_df.columns = [f'{col}_prev_month_1' for col in prev_month_1_df.columns]

# Объединение данных за предыдущие месяцы
prev_months_combined = prev_month_2_df.join(prev_month_1_df, how='outer')

# Объединение данных за последний месяц с данными за предыдущие месяцы
final_df = last_month_df.set_index('id').join(prev_months_combined, how='left')
# Вывод результата
final_df


Преобразование столбцов с информацией о кластерах в категориальный тип данных

In [ ]:
clusters = ['start_cluster_prev_month_1','start_cluster_prev_month_2','end_cluster_prev_month_1','end_cluster_prev_month_2']
final_df[clusters] = final_df[clusters].astype('category')

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

# Функция для добавления суффиксов
def add_suffix(cols, suffix):
    return [col + suffix for col in cols]

# Добавление суффиксов "_prev_month_2" и "_prev_month_1"
cat_cols = add_suffix(cat_cols, "_prev_month_2") + add_suffix(cat_cols, "_prev_month_1")
cat_cols = cat_cols+[
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",'start_cluster_prev_month_1','start_cluster_prev_month_2',
]
# Вывод результата
print(cat_cols)

 ### **Удаление незначимых признаков**:
   - Был проведен анализ нулевой важности признаков и выявлены незначимые признаки.
   - Незначимые признаки были удалены из данных для улучшения производительности модели и избежания переобучения.
   - Упрощение модели упростило оптимизацию параметров и дало прирост в точности и производительности.


In [ ]:
zero_importance = ['city_type',
 'cnt_cred_d_oper_1m',
 'sum_deb_d_oper_3m',
 'cnt_deb_d_oper_3m',
 'sum_cred_d_oper_3m',
 'cnt_days_cred_f_oper_3m',
 'cnt_days_cred_g_oper_3m',
 'cnt_cred_h_oper_3m',
 'city_prev_month_2',
 'city_type_prev_month_2',
 'ogrn_days_end_quarter_prev_month_2',
 'sum_of_paym_2m_prev_month_2',
 'cnt_b_oper_1m_prev_month_2',
 'cnt_days_cred_f_oper_1m_prev_month_2',
 'cnt_deb_g_oper_1m_prev_month_2',
 'cnt_days_cred_g_oper_1m_prev_month_2',
 'cnt_deb_h_oper_1m_prev_month_2',
 'cnt_cred_h_oper_1m_prev_month_2',
 'cnt_days_cred_h_oper_1m_prev_month_2',
 'sum_b_oper_3m_prev_month_2',
 'sum_cred_d_oper_3m_prev_month_2',
 'cnt_cred_d_oper_3m_prev_month_2',
 'sum_cred_f_oper_3m_prev_month_2',
 'cnt_days_cred_f_oper_3m_prev_month_2',
 'cnt_deb_g_oper_3m_prev_month_2',
 'cnt_cred_g_oper_3m_prev_month_2',
 'city_type_prev_month_1',
 'sum_a_oper_1m_prev_month_1',
 'cnt_b_oper_1m_prev_month_1',
 'sum_cred_d_oper_1m_prev_month_1',
 'cnt_cred_e_oper_1m_prev_month_1',
 'cnt_days_deb_f_oper_1m_prev_month_1',
 'cnt_days_cred_f_oper_1m_prev_month_1',
 'cnt_deb_g_oper_1m_prev_month_1',
 'cnt_days_deb_g_oper_1m_prev_month_1',
 'sum_cred_g_oper_1m_prev_month_1',
 'cnt_days_cred_h_oper_1m_prev_month_1',
 'cnt_deb_d_oper_3m_prev_month_1',
 'sum_cred_d_oper_3m_prev_month_1',
 'cnt_cred_f_oper_3m_prev_month_1']

In [ ]:
list(set(zero_importance) & set(cat_cols))

In [ ]:
cat_cols_to_remove = (list(set(zero_importance) & set(cat_cols)))
cat_cols_to_remove

In [ ]:
X = final_df.drop(["date", "end_cluster",'date_prev_month_1','date_prev_month_2','end_cluster_prev_month_1','end_cluster_prev_month_2'], axis=1)
X = X.drop(zero_importance, axis=1)
cat_cols = list(set(cat_cols).difference(cat_cols_to_remove))
y = final_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                   random_state=42)

## **Обучение модели**

В качестве базовой модели возьмем Catboost

In [ ]:
from catboost import CatBoostClassifier, cv, Pool

Зададим функцию для взвешенной метрики roc auc, которая вычисляет взвешенную площадь под ROC-кривой для многоклассовой классификации.

In [ ]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr")#, average=None)
    return sum(weights * classes_roc_auc)

 Загрузка весов кластеров из файла Excel и создание словаря weights_dict для использования в функции weighted_roc_auc

In [ ]:
cluster_weights = pd.read_excel("/kaggle/input/alphadataset/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

Проверка работы модели

 **Модель без учета весов**

Определение параметров модели CatBoost для обучения без учета весов

In [ ]:
cv_dataset = Pool(data=X,
                  label=y,
                  cat_features=cat_cols)

**Обучение модели CatBoost с кросс-валидацией с использованием параметров**

По кросс-валидации мы определяли оптимальные параметры модели( все параметры значимы), причем результаты не зависят от того, как мы разделим выборку.



In [ ]:
catboost_params = {
    'iterations': 1500,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'MultiClass',
    'task_type': 'GPU',  # Specify GPU usage
    'random_seed': 42,
    'early_stopping_rounds': 10
}

# Create CatBoost classifier
model0 = cv(cv_dataset, params=catboost_params, fold_count=4, return_models=True)

Визуализация распределения меток классов на обучающем и валидационном наборах данных

In [ ]:
sns.histplot(y_train)
sns.histplot(y_val)

#### Анализ гистограммы позволяет сделать вывод о преобладании класса alpha и класса {} в данных, что связано с распределением целевой переменной y. На гистограмме видно, что эти классы встречаются чаще других, что может привести к более высокому количеству ошибок при прогнозировании. Распределение классов не является равномерным, и класс{} и alpha можно выделить, как мажоритарные. Для улучшения качества модели можно добавить в распределение больше наблюдений о минорных классах.  Для решения этой проблемы мы пробовали различные методы балансировки классов, такие как oversampling (увеличение числа примеров меньшего класса), что не привело к желаемым результатам.


#### После попыток исправить дисбаланс классов и подстроить модель к бизнес-задаче с использованием весов, а также при экспериментах с изменением характера распределения или корректировки функции потерь через class_weights, качество модели не улучшилось, а скорее ухудшилось. Даже применение техник over-sampling и under-sampling не привело к желаемым результатам.


Повторное определение параметров модели CatBoost для обучения с учетом весов

In [ ]:
catboost_params = {
    'iterations': 1176,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'MultiClass',
    'task_type': 'GPU',  # Specify GPU usage
    'random_seed': 42
}

Обучение модели CatBoost с учетом весов, предсказание вероятностей и вычисление взвешенной ROC-AUC

In [ ]:
model0 = CatBoostClassifier(**catboost_params)

In [ ]:
model0.fit(X, y, cat_features=cat_cols, verbose=10)

In [ ]:
# Сохранение модели в файл
model0.save_model('catboost_model2.bin')

In [ ]:
y_pred_proba_train0 = model0.predict_proba(x_train)
weighted_roc_auc(y_train, y_pred_proba_train0, model0.classes_, weights_dict)

In [ ]:
y_pred_proba0 = model0.predict_proba(x_val)
weighted_roc_auc(y_val, y_pred_proba0, model0.classes_, weights_dict)

**Визуализация матрицы ошибок (Confusion Matrix) для базовой модели CatBoost**

In [ ]:
preds0 = model0.predict(x_train)
figure, ax = plt.subplots(figsize=(20, 13))
ConfusionMatrixDisplay.from_predictions(y_train, preds0, ax=ax)
plt.title('Confusion Matrix Base Catboost Model')
plt.show()

#### Анализируя confusion matrix, можно сделать вывод о том, что основная часть ошибок сосредоточена в классе alpha и {}. Это обусловлено тем, что в данных классах присутствует значительное количество данных, что усложняет задачу модели в правильной классификации. Такая неравномерность распределения данных приводит к снижению точности предсказаний .


Вычисление и визуализация важности признаков для базовой модели CatBoost

In [ ]:
feature_importances = pd.DataFrame(zip(x_train.columns, model0.feature_importances_))
feature_importances.columns = ['feature name', 'importance']
feature_importances

In [ ]:
figure, ax = plt.subplots(figsize=(20, 13))
sns.barplot(x = feature_importances['feature name'], y=feature_importances['importance'], ax=ax)
plt.title('Feature importance of Catboost. All features')
plt.show()

Выделяем подмножество наиболее важных признаков, которые имеют значение важности больше 1.

In [ ]:
very_important = feature_importances[feature_importances['importance'] > 1]

In [ ]:
very_important

 **Отображение наиболее важных признаков**

In [ ]:
figure, ax = plt.subplots(figsize=(20, 13))
sns.barplot(x = very_important['feature name'], y=very_important['importance'], ax=ax)
plt.title('Feature importance of Catboost. Top')
plt.show()

## **Прогноз на тестовой выборке**

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

**Фильтрация данных и объединение таблиц:**
   - Из тестовой выборки выделяются записи за последний месяц 'month_6' в last_month_df.
   - Создаются DataFrame для данных за предыдущие два месяца 'month_5' и 'month_4' в prev_month_5_df и prev_month_4_df.
   - Столбцы с категориальными признаками приводятся к типу 'category'.
   - Столбцы переименовываются, чтобы избежать конфликтов имен.
   - Данные за предыдущие месяцы объединяются в prev_months_combined.
   - Данные за последний месяц объединяются с данными за предыдущие месяцы в final_df.

In [ ]:
import pandas as pd

# Фильтрация данных, оставляем только записи за последний месяц
last_month_df = test_df[test_df['date'] == 'month_6']

# Создание DataFrame для данных за предыдущие два месяца
prev_month_5_df = test_df[test_df['date'] == 'month_5'].set_index('id')
prev_month_4_df = test_df[test_df['date'] == 'month_4'].set_index('id')
prev_month_5_df[cat_cols] = prev_month_5_df[cat_cols].astype("category")
prev_month_4_df[cat_cols] = prev_month_4_df[cat_cols].astype("category")

# Переименование столбцов, чтобы избежать конфликтов имен
prev_month_5_df.columns = [f'{col}_prev_month_2' for col in prev_month_5_df.columns]
prev_month_4_df.columns = [f'{col}_prev_month_1' for col in prev_month_4_df.columns]

# Объединение данных за предыдущие месяцы
prev_months_combined = prev_month_5_df.join(prev_month_4_df, how='outer')

# Объединение данных за последний месяц с данными за предыдущие месяцы
final_df = last_month_df.set_index('id').join(prev_months_combined, how='left')

# Вывод результата
final_df


In [ ]:
test_df.pivot(index="id", columns="date", values="start_cluster").head(3)

**Заполнение стартового кластера:**

Для того, чтобы сделать прогноз на тестовой выборке, нужно заполнить стартовый кластер. </br>
В качестве базового подхода заполним все стартовые кластеры, самым популярным кластером.

In [ ]:
filtered_rows = test_df[test_df['date'] == 'month_5']
start_cluster_column = filtered_rows['start_cluster']

# Вывод результатов
print(start_cluster_column)

In [ ]:
sample_submission_df = pd.read_csv("/kaggle/input/alphadataset/sample_submission.csv")

In [ ]:
sample_submission_df.shape

In [ ]:
sample_submission_df.head()

Для тестовой выборки будем использовать только последний месяц

### **Подготовка данных для прогноза:**
   - Удаляются лишние столбцы и добавляется стартовый кластер.
   - Категориальные признаки указываются в списке cat_cols.
  
   - Удаляются ненужные столбцы и категориальные признаки, которые необходимо удалить.
   - Пропуски в столбцах с кластерами заполняются значением "None".

In [ ]:
final_df

In [ ]:
final_df = final_df.drop(["date",'date_prev_month_1','date_prev_month_2'], axis=1)
final_df.drop("start_cluster", axis=1)
final_df["start_cluster"] = start_cluster_column.values
final_df["start_cluster"] = final_df["start_cluster"].astype("category")
final_df["start_cluster"]

In [ ]:
final_df

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

# Функция для добавления суффиксов
def add_suffix(cols, suffix):
    return [col + suffix for col in cols]

# Добавление суффиксов "_prev_month_2" и "_prev_month_1"
cat_cols = add_suffix(cat_cols, "_prev_month_2") + add_suffix(cat_cols, "_prev_month_1")
cat_cols = cat_cols+[
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]
# Вывод результата
print(cat_cols)

In [ ]:
cat_cols = list(set(cat_cols).difference(cat_cols_to_remove))

In [ ]:
final_df = final_df.drop(zero_importance, axis=1)

In [ ]:
clusters = ['start_cluster_prev_month_1','start_cluster_prev_month_2']
final_df[clusters] = final_df[clusters].astype('category')

In [ ]:
final_df[cat_cols]

In [ ]:
final_df['start_cluster_prev_month_1'] = final_df['start_cluster_prev_month_1'].cat.add_categories(["None"])
final_df['start_cluster_prev_month_2'] = final_df['start_cluster_prev_month_2'].cat.add_categories(["None"])

In [ ]:
#final_df[final_df[cat_cols]].fillna("None".astype('category'), inplace=True)
final_df[final_df.columns[158]].fillna("None", inplace=True)
final_df[final_df.columns[159]].fillna("None", inplace=True)
final_df[final_df.columns[160]].fillna("None", inplace=True)
final_df[final_df.columns[163]].fillna("None", inplace=True)
final_df[final_df.columns[164]].fillna("None", inplace=True)
final_df[final_df.columns[169]].fillna("None", inplace=True)
final_df[final_df.columns[170]].fillna("None", inplace=True)
final_df[final_df.columns[229]].fillna("None", inplace=True)

### **Прогноз с помощью модели:**


In [ ]:
test_pred_proba = model0.predict_proba(final_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model0.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [ ]:
test_pred_proba_df.shape

In [ ]:
test_pred_proba_df.head(2)

### **Создание файла с прогнозом:**

In [ ]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("submission_final.csv", index=False)

In [ ]:
sample_submission_df